## Step 3: Conduct a vector similarity search with the Prompt and Send the Final Prompt to the LLM

In this notebook, we will first conduct a similarity search in the ChromaDB for texts which exhibit the least distance from the prompt created in step 02. 10 pieces of text will selected, but this number can be changed as desired. 

Subsequently, the final prompt (which includes both the prompt and the found results) will be sent to an LLM on AWS Bedrock. 

> Credit for parts of this notebook: https://github.com/Farzad-R/Advanced-QA-and-RAG-Series/commits?author=Farzad-R

In [ ]:
import chromadb
from pyprojroot import here

# Load ChromaDB
chroma_client = chromadb.PersistentClient(path=str(here("data/chroma")))
vectordb = chroma_client.get_collection(name="renalworks")
vectordb.count()

In [ ]:
import os
from src.utils.bedrock_caller import BedrockCaller

bedrock_caller = BedrockCaller()

# Specify the path of the prompt
prompt_file_name = 'clinical-notes-01-prompt-no-sample.txt'
prompt_file_path = os.path.join('prompts', prompt_file_name)

# Load prompt
with open(prompt_file_path, 'r') as f:
    prompt = f.read()

# Obtain prompt embedding
prompt_embedding = bedrock_caller.get_embedding(prompt)
print(prompt_embedding)

In [ ]:
# Conduct a similarity search in the ChromaDB
similarity_search_results = vectordb.query(
    query_embeddings = prompt_embedding,
    n_results=20 # Adjust the number of results to obtain as required 
)['documents']

print(similarity_search_results)

### Construction of Final Prompt & Response Generation

The final prompt is constructed as with the prompt from step 02 and the similarity search results obtained from above, before being sent to Anthropic's Claude 3 Sonnet for response generation. 

In [ ]:
sample_file_name = 'clinical-notes-01.txt'
sample_file_path = os.path.join('sample_responses', sample_file_name)

# Load sample
with open(sample_file_path, 'r') as f:
    sample = f.read()

system_role = "You are a highly knowledgeable assistant specialized in creating clinical documents and answering questions. You will receive the user's question along with the search results of that question over a database. Please use the search results and follow the sample's format to generate the proper answer. Please use JSON format. "
final_prompt = f"User's question: {prompt} \n\n Search results:\n {similarity_search_results} \n\n Sample: {sample}"

messages_API_body = {
    "anthropic_version": "bedrock-2023-05-31", 
    "max_tokens": int(500/0.75),
    "system": system_role,
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": final_prompt
                }
            ]
        }
    ]
}

In [ ]:
response_body = bedrock_caller.call_claude3sonnet(messages_API_body)
llm_output = response_body['content'][0]['text']

print(llm_output)

The LLM output is then saved into the `results` folder. 

In [ ]:
import re
import os

sample_file_name = 'clinical-notes-01.txt'
save_dir = os.path.join('results', os.path.splitext(sample_file_name)[0])

# Regular expression to match files named 'attempt-x.txt'
pattern = re.compile(r'attempt-(\d+)\.txt')
max_num = -1

# Find the highest attempt number
for filename in os.listdir(save_dir):
    match = pattern.match(filename)
    if match:
        num = int(match.group(1))
        if num > max_num:
            max_num = num
next_num = max_num + 1 if max_num != -1 else 1

# Create the new filename
new_filename = f'attempt-{next_num}.txt'
new_filepath = os.path.join(save_dir, new_filename)

# Save the data to the new file
with open(new_filepath, 'w') as file:
    file.write(llm_output)